In [1]:
import cobra

In [2]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_annotated_curated_with_phenol.xml")

In [3]:
model

Name,ropacus_annotated_curated
Memory address,0x0138f5a850
Number of metabolites,1583
Number of reactions,2383
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


In [39]:
for reaction in model.reactions[:50]:
    if reaction.gene_reaction_rule:
        print(reaction.gene_reaction_rule)

WP_187300246_1
WP_187300246_1
( WP_005248578_1 and WP_025432775_1 ) or ( WP_005248578_1 and WP_025433613_1 ) or ( WP_005249637_1 and WP_025432775_1 )
WP_005248999_1
WP_005248999_1
WP_005248999_1
WP_005248999_1
WP_005248999_1
WP_005248999_1
WP_005248999_1
WP_005248999_1
WP_005248999_1
WP_005248999_1
WP_005248999_1
WP_005248999_1
WP_005248999_1
WP_005248999_1
WP_025433301_1
WP_005246696_1
WP_005244822_1
WP_005244822_1
WP_080578199_1
WP_005250095_1 or WP_005252000_1
WP_005239747_1 or WP_005245250_1


In [14]:
for m in model.metabolites:
    print(m.name)
    if 'glucose' in m.name:
        print(m)

idylglycerol (ditetradecanoyl, n-C14:0)
1,2-ditetradec-7-enoyl-sn-glycerol 3-phosphate
Phosphatidylglycerol (ditetradec-7-enoyl, n-C14:1)
1,2-dihexadecanoyl-sn-glycerol 3-phosphate
1,2-dihexadec-9-enoyl-sn-glycerol 3-phosphate
Phosphatidylglycerol (dihexadec-9-enoyl, n-C16:1)
1,2-dioctadecanoyl-sn-glycerol 3-phosphate
Phosphatidylglycerol (dioctadecanoyl, n-C18:0)
1,2-dioctadec-11-enoyl-sn-glycerol 3-phosphate
Phosphatidylglycerol (dioctadec-11-enoyl, n-C18:1)
Fe3+
Cytosine
CMP
CMP
CO2
Coniferol
Coniferyl aldehyde
Ferulate
Co2+
Ferredoxin (oxidized) 2[4Fe-4S]
Ferredoxin (reduced) 2[4Fe-4S]
Undecaprenyl phosphate
Coniferol
Coniferol
Carbon monoxide
Cyclic pyranopterin monophosphate
Coproporphyrinogen III
Protoporphyrinogen IX
P-Cresol
P-Cresol
L-Carnitine
L-Carnitine
L-Carnitinyl-CoA
Gamma-butyrobetaine
Crotonobetaine
Crotonobetainyl-CoA
D-Carnitine
D-Carnitine
Gamma-butyrobetaine
Chromate
Chromate
Uracil
Cytosine
Spermidine
Crotonobetaine
Crotonobetaine
Tetradecenoyl-CoA (n-C14:1CoA)
U

In [21]:
model.reactions.get_by_id('PGI').metabolites

{<Metabolite g6p_c at 0x139060c50>: -1.0,
 <Metabolite f6p_c at 0x1390f4110>: 1.0}

In [67]:
for r in model.metabolites.get_by_id('2pg_c').reactions:
    print(r)

ENO: 2pg_c <=> h2o_c + pep_c
PGM: 2pg_c <=> 3pg_c
PGM_1: 3pg_c <=> 2pg_c
GLYCK2: atp_c + glyc__R_c --> 2pg_c + adp_c + h_c


In [80]:
met_1 = model.metabolites.get_by_id('pep_c')
met_2 = model.metabolites.get_by_id('pyr_c')

forward_string = ''
reverse_string = ''
for r in met_1.reactions:
    if met_2 in r.metabolites:
        if met_1 in r.reactants:
            forward_string += r.id + ' or '
            reverse_string += r.reverse_id + ' or '
            # print(r)
        else:
            forward_string += r.reverse_id + ' or '
            reverse_string += r.id + ' or '

print(forward_string)
    # for m in r.metabolites:
    #     print(m.name)
        # if m.name == 'g6p_c':
        # # print(m)
        #     print(r.metabolites)
        #     print()

FRUpts2pp or SBTpts or MNLpts or ACGApts or FRUpts2 or MANpts or FRUpts or ARBTpts or PYK or GALTpts or SUCpts or TREptspp or TREpts or GLCptspp or PPS_reverse_1c319 or ACGAptspp or SALCptspp or MALTptspp or FRUptspp or SUCptspp or DHAPT or MALTpts or ACMANApts or GAMpts or SALCpts or GLCpts or 


In [68]:
model.reactions.get_by_id('ENO').reverse_id

'ENO_reverse_40eea'

In [ ]:
metabolite_name = ''
reactions = [r for r in model.reactions if 